In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ximo/GitHub/skforecast


In [3]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

regressor = CatBoostRegressor()

In [4]:
type(regressor).__name__

'CatBoostRegressor'

In [13]:
def set_cpu_gpu_device(
    regressor: object, 
    device: str | None = 'cpu'
) -> str:
    """
    Set the device for the regressor to either 'cpu', 'gpu', 'cuda', or None.
    
    Parameters
    ----------
    regressor : object
        An estimator object from XGBoost, LightGBM, or CatBoost.
    device : str, default 'cpu'
        The device to set. Can be 'cpu', 'gpu', or None. If None, it will
        use the current device of the regressor.
        
    Returns
    -------
    original_device : str
        The original device of the regressor before setting it to the new device.

    """

    device_names = {
        'XGBRegressor': 'device',
        'LGBMRegressor': 'device',
        'CatBoostRegressor': 'task_type',
    }

    device_values = {
        'XGBRegressor': {'gpu': 'cuda', 'cpu': 'cpu', 'cuda': 'cuda'},
        'LGBMRegressor': {'gpu': 'gpu', 'cpu': 'cpu', 'cuda': 'gpu'},
        'CatBoostRegressor': {'gpu': 'GPU', 'cpu': 'CPU', 'cuda': 'GPU'},
    }

    if device not in ['gpu', 'cpu', 'cuda', None]:
        raise ValueError("`device` must be 'gpu', 'cpu', 'cuda', or None.")

    regressor_name = type(regressor).__name__
    if regressor_name not in device_names:
        return None
    
    original_device = regressor.get_params()[device_names[regressor_name]].lower()

    if device is None:
        return original_device
    
    param_name = device_names[regressor_name]
    new_device = device_values[regressor_name][device]

    if original_device != new_device:
        try:
            regressor.set_params(**{param_name: new_device})
        except Exception:
            pass

    return original_device

In [14]:
import pytest
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression


@pytest.mark.parametrize("regressor, initial_device, new_device, expected_new_device", [
    (XGBRegressor(), "cpu", "gpu", "cuda"),
    (XGBRegressor(), "cuda", "cpu", "cpu"),
    (LGBMRegressor(), "gpu", "cpu", "cpu"),
    (LGBMRegressor(), "cpu", "gpu", "gpu"),

])
def test_set_cpu_gpu_device_changes_device(regressor, initial_device, new_device, expected_new_device):
    regressor.set_params(**{'device': initial_device})
    original = set_cpu_gpu_device(regressor, new_device)
    assert original.lower() == initial_device.lower()
    assert regressor.get_params()['device'] == expected_new_device

def test_set_cpu_gpu_device_no_change_if_same():
    regressor = XGBRegressor(device="cuda")
    _ = set_cpu_gpu_device(regressor, "cuda")
    assert regressor.get_params()['device'] == "cuda"

def test_set_cpu_gpu_device_invalid_device():
    regressor = LGBMRegressor()
    msg = "device must be 'gpu', 'cpu', or None."
    with pytest.raises(ValueError, match=msg):
        set_cpu_gpu_device(regressor, "tpu")

def test_set_cpu_gpu_device_unsupported_model_returns_none():
    regressor = LinearRegression()
    result = set_cpu_gpu_device(regressor, "cpu")
    assert result is None

def test_set_cpu_gpu_device_none_device_returns_current():
    regressor = XGBRegressor(device="cuda")
    original = set_cpu_gpu_device(regressor, None)
    assert original == "cuda"


In [15]:
regressor = XGBRegressor(device="cuda")
original = set_cpu_gpu_device(regressor, None)
original

'cuda'

In [5]:
regressor = CatBoostRegressor(task_type='GPU')
print(regressor.get_params()['task_type'])
original_device = set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['task_type'])
set_cpu_gpu_device(regressor, original_device)
print(regressor.get_params()['task_type'])

GPU
CPU
GPU


In [6]:
regressor = XGBRegressor(device='cuda')
print(regressor.get_params()['device'])
original_device = set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, original_device)
print(regressor.get_params()['device'])

cuda
cpu
cuda


In [7]:
regressor = LGBMRegressor(device='gpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, 'gpu')
print(regressor.get_params()['device'])

gpu
cpu
gpu
